<a href="https://colab.research.google.com/github/danjshaw/ece57000-finalProject/blob/main/lora-bert-tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install datasets
!pip install evaluate
!pip freeze > requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.2 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import numpy as np
from transformers import set_seed
set_seed(0)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Followed examples from this [Hugging Face NLP course](https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt#fine-tuning-a-model-with-the-trainer-api) on how to use the trainer API for fine-tuning.

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [5]:
import evaluate

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [6]:
num_runs = 5

Hyperparameters from [google-research/bert](https://github.com/google-research/bert):



> For each task, we selected the best fine-tuning hyperparameters from the lists below, and trained for 4 epochs:
> * batch sizes: 8, 16, 32, 64, 128
> * learning rates: 3e-4, 1e-4, 5e-5, 3e-5



In [7]:
epochs = 4
batch_sizes = [8, 16, 32, 64, 128]
learning_rates = [3e-4, 1e-4, 5e-5, 3e-5]

# Full Fine-Tuning



In [8]:
from transformers import AutoModelForSequenceClassification

ft_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
import time

start_time = time.time()

ft_results = []
iterations = len(batch_sizes)*len(learning_rates)*num_runs
progress = 0
for run in range(1, num_runs+1):
  for size in batch_sizes:
    for rate in learning_rates:
      progress += 1
      print(f"Progress: {progress}/{iterations}")
      ft_result = {"run":run, "batch_size": size, "learning_rate": rate}

      ft_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

      training_args = TrainingArguments(
          "ft-trainer",
          eval_strategy="epoch",
          per_device_eval_batch_size=size,
          per_device_train_batch_size=size,
          num_train_epochs=epochs,
          learning_rate=rate,
          disable_tqdm=True,
          report_to="none"
      )

      trainer = Trainer(
          ft_model,
          training_args,
          train_dataset=tokenized_datasets["train"],
          eval_dataset=tokenized_datasets["validation"],
          data_collator=data_collator,
          processing_class=tokenizer,
          compute_metrics=compute_metrics
      )

      ft_result = ft_result | trainer.train().metrics | trainer.evaluate()
      ft_results.append(ft_result)

end_time = time.time()

runtime_seconds = end_time - start_time
runtime_minutes = runtime_seconds / 60

ft_max = ft_results[0]
for i, result in enumerate(ft_results):
  if result['eval_f1'] > ft_max['eval_f1']:
    ft_max = result

print(f'\n================ \
        \nRuntime: {runtime_minutes} minutes \
        \nBest Result: \
        \nbatch_size={ft_max["batch_size"]}, \
        \nlearning_rate={ft_max["learning_rate"]}, \
        \neval_f1={ft_max["eval_f1"]}')

import csv

with open('/content/drive/MyDrive/lora-bert-tiny/ft-results.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=list(ft_results[0].keys()))

    writer.writeheader()
    for result in ft_results:
      writer.writerow(result)

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Progress: 1/100


{'eval_loss': 0.5835773944854736, 'eval_accuracy': 0.7181372549019608, 'eval_f1': 0.8275862068965517, 'eval_runtime': 0.6544, 'eval_samples_per_second': 623.449, 'eval_steps_per_second': 77.931, 'epoch': 1.0}
{'loss': 0.5911, 'grad_norm': 3.7462449073791504, 'learning_rate': 0.00021830065359477123, 'epoch': 1.0893246187363834}
{'eval_loss': 0.5930801630020142, 'eval_accuracy': 0.7132352941176471, 'eval_f1': 0.8163265306122449, 'eval_runtime': 0.4871, 'eval_samples_per_second': 837.676, 'eval_steps_per_second': 104.71, 'epoch': 2.0}
{'loss': 0.4593, 'grad_norm': 22.534181594848633, 'learning_rate': 0.00013660130718954247, 'epoch': 2.178649237472767}
{'eval_loss': 0.8781757950782776, 'eval_accuracy': 0.696078431372549, 'eval_f1': 0.7905405405405406, 'eval_runtime': 0.5017, 'eval_samples_per_second': 813.258, 'eval_steps_per_second': 101.657, 'epoch': 3.0}
{'loss': 0.283, 'grad_norm': 0.7782564163208008, 'learning_rate': 5.4901960784313716e-05, 'epoch': 3.2679738562091503}
{'eval_loss': 1

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

{'eval_loss': 0.5759289264678955, 'eval_accuracy': 0.7352941176470589, 'eval_f1': 0.8291139240506329, 'eval_runtime': 0.385, 'eval_samples_per_second': 1059.874, 'eval_steps_per_second': 10.391, 'epoch': 4.0}
Progress: 98/100
{'eval_loss': 0.6038169264793396, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.3483, 'eval_samples_per_second': 1171.479, 'eval_steps_per_second': 11.485, 'epoch': 1.0}
{'eval_loss': 0.5951903462409973, 'eval_accuracy': 0.7058823529411765, 'eval_f1': 0.8224852071005917, 'eval_runtime': 0.4502, 'eval_samples_per_second': 906.283, 'eval_steps_per_second': 8.885, 'epoch': 2.0}
{'eval_loss': 0.579287052154541, 'eval_accuracy': 0.6911764705882353, 'eval_f1': 0.8090909090909091, 'eval_runtime': 0.4939, 'eval_samples_per_second': 826.086, 'eval_steps_per_second': 8.099, 'epoch': 3.0}
{'eval_loss': 0.5748432874679565, 'eval_accuracy': 0.6911764705882353, 'eval_f1': 0.8073394495412844, 'eval_runtime': 0.3847, 'eval_samples_per_secon

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.5748432874679565, 'eval_accuracy': 0.6911764705882353, 'eval_f1': 0.8073394495412844, 'eval_runtime': 0.2925, 'eval_samples_per_second': 1394.987, 'eval_steps_per_second': 13.676, 'epoch': 4.0}
Progress: 99/100
{'eval_loss': 0.6207587718963623, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.4565, 'eval_samples_per_second': 893.679, 'eval_steps_per_second': 8.762, 'epoch': 1.0}
{'eval_loss': 0.6050003170967102, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.3788, 'eval_samples_per_second': 1077.16, 'eval_steps_per_second': 10.56, 'epoch': 2.0}
{'eval_loss': 0.5970742106437683, 'eval_accuracy': 0.6887254901960784, 'eval_f1': 0.8145985401459854, 'eval_runtime': 0.4338, 'eval_samples_per_second': 940.524, 'eval_steps_per_second': 9.221, 'epoch': 3.0}
{'eval_loss': 0.5952083468437195, 'eval_accuracy': 0.6936274509803921, 'eval_f1': 0.8169838945827232, 'eval_runtime': 0.3855, 'eval_samples_per_secon

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.5952083468437195, 'eval_accuracy': 0.6936274509803921, 'eval_f1': 0.8169838945827232, 'eval_runtime': 0.388, 'eval_samples_per_second': 1051.643, 'eval_steps_per_second': 10.31, 'epoch': 4.0}
Progress: 100/100
{'eval_loss': 0.6439599990844727, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.3439, 'eval_samples_per_second': 1186.531, 'eval_steps_per_second': 11.633, 'epoch': 1.0}
{'eval_loss': 0.6185536980628967, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.3789, 'eval_samples_per_second': 1076.742, 'eval_steps_per_second': 10.556, 'epoch': 2.0}
{'eval_loss': 0.6116704344749451, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.4408, 'eval_samples_per_second': 925.545, 'eval_steps_per_second': 9.074, 'epoch': 3.0}
{'eval_loss': 0.609874427318573, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.3688, 'eval_samples_per_sec

# Low-Rank Adaptation (LoRA)

In [10]:
class LoraModule(nn.Module):
  def __init__(self, in_features, out_features, rank, alpha):
    super().__init__()
    self.scale = alpha / rank
    self.A = nn.Parameter(torch.randn(in_features, rank))
    self.B = nn.Parameter(torch.zeros(rank, out_features))

  def forward(self, x):
    return (self.scale * (x @ self.A @ self.B))

class LoraLinear(nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    if (isinstance(linear, LoraLinear)):
      self.linear = linear.linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

    else:
      self.linear = linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

  def forward(self, x):
    return self.linear(x) + self.lora(x)

In [11]:
def get_trainable_parameters(model):
  trainable_parameters = 0
  parameters = 0
  for param in model.parameters():
    parameters += param.numel()
    if param.requires_grad:
      trainable_parameters += param.numel()
  return parameters, trainable_parameters

In [12]:
def configure_lora_model(model, rank, alpha):
  for i, layer in enumerate(model.bert.encoder.layer):
    s = layer.attention.self
    s.query = LoraLinear(s.query, rank, alpha)
    s.value = LoraLinear(s.value, rank, alpha)

  for name, param in model.named_parameters():
    if 'A' in name or 'B' in name:
      param.requires_grad = True
    else:
      param.requires_grad = False
  params, trainable_params = get_trainable_parameters(model)
  print(f"Total Parameters={params}, Trainable Parameters={trainable_params}")

  return (params, trainable_params)

In [13]:
from transformers import AutoModelForSequenceClassification

lora_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
ranks = [1, 2, 4, 8, 16]
alphas = [1, 2, 4, 8, 16]

In [ ]:
from transformers import Trainer, TrainingArguments
import time

start_time = time.time()

results = []
iterations = len(batch_sizes)*len(learning_rates)*len(ranks)*len(alphas)*num_runs
progress = 0
for run in range(1, num_runs+1):
  for rank in ranks:
    for alpha in alphas:
      for size in batch_sizes:
        for rate in learning_rates:
          progress += 1
          print(f"Progress: {progress}/{iterations}")
          result = {"run": run, "rank": rank, "alpha": alpha, "batch_size": size, "learning_rate": rate}

          params, trainable_params = configure_lora_model(lora_model, rank, alpha)
          result["parameters"] = params
          result["trainable_parameters"] = trainable_params

          training_args = TrainingArguments(
              "lora-trainer",
              eval_strategy="epoch",
              per_device_eval_batch_size=size,
              per_device_train_batch_size=size,
              num_train_epochs=epochs,
              learning_rate=rate,
              disable_tqdm=True,
              report_to="none"
          )

          trainer = Trainer(
              lora_model,
              training_args,
              train_dataset=tokenized_datasets["train"],
              eval_dataset=tokenized_datasets["validation"],
              data_collator=data_collator,
              processing_class=tokenizer,
              compute_metrics=compute_metrics
          )

          result = result | trainer.train().metrics | trainer.evaluate()
          results.append(result)

end_time = time.time()

runtime_seconds = end_time - start_time
runtime_hours = runtime_seconds / 3600

max = results[0]
for i, result in enumerate(results):
  if result['eval_f1'] > max['eval_f1']:
    max = result

print(f'\n================ \
        \nRuntime: {runtime_hours} hours \
        \nBest Result: \
        \nrank={max["rank"]}, \
        \nalpha={max["alpha"]}, \
        \nparameters={max["parameters"]}, \
        \ntrainable_parameters={max["trainable_parameters"]}, \
        \nbatch_size={max["batch_size"]}, \
        \nlearning_rate={max["learning_rate"]}, \
        \neval_f1={max["eval_f1"]}')

import csv

with open('/content/drive/MyDrive/lora-bert-tiny/lora-results.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=list(results[0].keys()))

    writer.writeheader()
    for result in results:
      writer.writerow(result)

Progress: 1/2500
Total Parameters=4387202, Trainable Parameters=1024
{'eval_loss': 0.6257390975952148, 'eval_accuracy': 0.6813725490196079, 'eval_f1': 0.8104956268221575, 'eval_runtime': 0.5656, 'eval_samples_per_second': 721.355, 'eval_steps_per_second': 90.169, 'epoch': 1.0}
{'loss': 0.663, 'grad_norm': 0.46347522735595703, 'learning_rate': 0.00021830065359477123, 'epoch': 1.0893246187363834}
{'eval_loss': 0.6210986375808716, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.591, 'eval_samples_per_second': 690.353, 'eval_steps_per_second': 86.294, 'epoch': 2.0}
{'loss': 0.6298, 'grad_norm': 0.40943172574043274, 'learning_rate': 0.00013660130718954247, 'epoch': 2.178649237472767}
{'eval_loss': 0.6197056770324707, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.6239, 'eval_samples_per_second': 653.954, 'eval_steps_per_second': 81.744, 'epoch': 3.0}
{'loss': 0.6246, 'grad_norm': 0.42999500036239624, 'learning_rate